# IHT Experiments to verify theory L-0 Norm - F-MNIST

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import numpy as np
from scipy.fftpack import dct,idct
from keras.datasets import mnist, fashion_mnist
from PIL import Image


sys.path.append('../')

from models.util import *


#Seed used for choosing classes, training points, and test points.
#SEED = 14
SEED=11

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_samples = 500
sqrt_n = 28
input_shape=(sqrt_n,sqrt_n,1)
n = sqrt_n*sqrt_n
k = 4
c=2.0

In [3]:
#Load F-MNIST data
(X_train, _), (X_test, _) = fashion_mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

fm_data = np.concatenate((X_train,X_test),axis=0)

In [4]:
#Normalize the data
fm_data = fm_data/255.0

In [5]:
#Check FMNIST results for 1000 random images
#Check Cifar10 results for 500 random images - BP
t = int(float(n)/27.0/k/c)
print('Noise budget of %d' % t)
subset_idx = np.random.choice(np.arange(fm_data.shape[0]),num_samples)
fm_data_sub = fm_data[subset_idx]
fm_data_y = np.zeros((num_samples,sqrt_n,sqrt_n))
ft_values = np.zeros(num_samples)
for i in range(num_samples):
    #first sample an element from the data
    x = fm_data_sub[i,:,:,0].flatten()
    #Now sample a t - must be atleast 1
    t_l = np.random.randint(1,t)
    ft_values[i] = t_l
    #Now samnple the an index set from [n] with cardinality = t_l
    s = np.random.choice(np.arange(n),t_l)
    e = np.zeros(n)
    #Now create the vector e
    #pick a value for each element between 0 and 1 as the images are normalized
    for j in range(t_l):
        e[s[j]] = np.random.uniform()
    y = x + e
    fm_data_y[i,:,:] = y.reshape((sqrt_n,sqrt_n))        

Noise budget of 3


# Get \delta_p and \Delta_p

In [6]:
%%capture two
ferrors_l2 = np.zeros(fm_data_y.shape[0])
ferrors_inf = np.zeros(fm_data_y.shape[0])
fbot_l2 = np.zeros(fm_data_y.shape[0])
ftau = np.zeros(fm_data_y.shape[0])
fdiff_l2 = np.zeros(fm_data_y.shape[0])
fdiff_inf = np.zeros(fm_data_y.shape[0])
fT_values = np.zeros(fm_data_y.shape[0])

for i in range(num_samples):
    y = fm_data_y[i,:,:].flatten()
    x = fm_data_sub[i,:,:].flatten()
    
    #Get actual top k and bottom k
    x_hat_top_k, x_hat_bot_k =  get_top_bot_k_vec(dct(x, norm='ortho'),k=k)
    #Now we need to calculate T
    e = y - x 
    #Get the constant that we need to set to zero as a function of T
    a = np.sqrt(np.linalg.norm(x_hat_top_k)**2 + np.linalg.norm(e)**2)
    #Lets get rho
    rho = np.sqrt(27)*np.sqrt((c*k*ft_values[i])/float(n))
    
    #We want to find a T such that rho^T a is very small - we use 1e-20 
    T = int((np.log(1e-20) - np.log(a))/np.log(rho))
    fT_values[i] = T
    #Get top k approx
    x_hat_approx,e_hat_approx = iht(y,int(ft_values[i]),T=T,k=k)
    
    #Note the errors
    ferrors_l2[i] = np.linalg.norm(x_hat_top_k.flatten()- x_hat_approx.flatten())
    ferrors_inf[i] = np.linalg.norm(x_hat_top_k.flatten()- x_hat_approx.flatten(), ord=np.inf)

    #Note the norm of bottom k coefficients
    fbot_l2[i] = np.linalg.norm(x_hat_bot_k)

    #Get the multiplicative constant
    c_l2 = np.sqrt((4*k*c*ft_values[i])/n)
    c_inf = np.sqrt((2*c*ft_values[i])/n)
    #Calculate tau
    ftau[i] = (np.sqrt(3)*np.sqrt(1 + 2*np.sqrt((k*c*ft_values[i])/n))/(1-rho))
    
    #Calculate the difference from the upper bound
    fdiff_l2[i] = (c_l2*ftau[i]*fbot_l2[i]) - ferrors_l2[i] 
    fdiff_inf[i] = (c_inf*ftau[i]*fbot_l2[i]) - ferrors_inf[i]

In [7]:
print(np.mean(ft_values), 
      np.mean(ftau), 
      np.mean(ferrors_l2), 
      np.mean(fdiff_l2), 
      np.mean(ferrors_inf), 
      np.mean(fdiff_inf))

1.468 5.693143134158139 0.1434396591047507 8.273905988923092 0.10299919985554525 2.8729818938002363


In [8]:
fmnist_tup = (fm_data_y, fm_data_sub, ft_values,ftau,ferrors_l2, ferrors_inf, fdiff_l2, fdiff_inf )

import pickle

with open('data/fmnist_tuple_theory.pickle', 'wb') as f:
    pickle.dump(fmnist_tup, f)